# Pre-trained model: Microsoft/phi-2

### Loading & cleaning

In [45]:
# Imports
import os
import pandas as pd

# Load data
data = '../data/ready_for_phi-2/'

print(os.path.exists(data))
print(os.listdir(data))

True
['test_samples_1.csv', 'test_samples_2.csv', 'train_samples_1.csv', 'train_samples_2.csv']


In [46]:
# Load
train10k = pd.read_csv(data + 'train_samples_1.csv')
train5k = pd.read_csv(data + 'train_samples_2.csv')

test2k = pd.read_csv(data + 'test_samples_1.csv')
test1k = pd.read_csv(data + 'test_samples_2.csv')

train10k.shape, train5k.shape, test2k.shape, test1k.shape

((9999, 7), (4998, 7), (1998, 7), (999, 7))

In [47]:
train10k.head()

,label,text,clean_text,word_count,emotions,prompt,response
0,2,The cashier told my husband that should he get...,the cashier told my husband that should he get...,85,"annoyance, anger, neutral",[INST] A customer left us a 2-star review: 'th...,e customer's experience and address their conc...
1,2,Here's what happened:\n- We were suppose to be...,here's what happened n we were suppose to be a...,297,"realization, neutral, disappointment",[INST] A customer left us a 2-star review: 'he...,"e customer's experience, the following steps c..."
2,2,Went to this Fest today and admission is $12. ...,went to this fest today and admission is 12. t...,197,"neutral, disapproval, approval",[INST] A customer left us a 2-star review: 'we...,"e customer's experience, we can focus on the f..."
3,2,went to check out this joint the other weekend...,went to check out this joint the other weekend...,82,"admiration, approval, neutral",[INST] A customer left us a 2-star review: 'we...,"e customer's experience, we could focus on mod..."
4,2,A whole lot of meh. This store is in a horribl...,a whole lot of meh. this store is in a horribl...,34,"disappointment, sadness, fear",[INST] A customer left us a 2-star review: 'a ...,"e customer's experience, we can address the fo..."


In [48]:
train10k['label'].value_counts(), train5k['label'].value_counts(),test2k['label'].value_counts(), test1k['label'].value_counts()
# We are good on balance.

(label
 2    3333
 1    3333
 3    3333
 Name: count, dtype: int64,
 label
 2    1666
 1    1666
 3    1666
 Name: count, dtype: int64,
 label
 1    666
 3    666
 2    666
 Name: count, dtype: int64,
 label
 1    333
 3    333
 2    333
 Name: count, dtype: int64)

In [49]:
# Selecting only the columns we need

# 15k training samples
train10k = train10k[['prompt', 'response']]
train5k = train5k[['prompt', 'response']]

# 3k test samples
test2k = test2k[['prompt', 'response']]
test1k = test1k[['prompt', 'response']]

# Rename columns
train10k.columns = ['input', 'output']
train5k.columns = ['input', 'output']
test2k.columns = ['input', 'output']
test1k.columns = ['input', 'output']

# Check
train10k.shape, train5k.shape, test2k.shape, test1k.shape

((9999, 2), (4998, 2), (1998, 2), (999, 2))

In [50]:
train10k.head()

,input,output
0,[INST] A customer left us a 2-star review: 'th...,e customer's experience and address their conc...
1,[INST] A customer left us a 2-star review: 'he...,"e customer's experience, the following steps c..."
2,[INST] A customer left us a 2-star review: 'we...,"e customer's experience, we can focus on the f..."
3,[INST] A customer left us a 2-star review: 'we...,"e customer's experience, we could focus on mod..."
4,[INST] A customer left us a 2-star review: 'a ...,"e customer's experience, we can address the fo..."


In [51]:
# Cleaning function for the outputs
import re

# Find "1" and remove the preceding chars, making strings start with "1"
def remove_before_one(text):
    index = text.find("1.")
    return text[index:] if index != -1 else text

In [52]:
train10k['output'] = train10k['output'].apply(remove_before_one)
train5k['output'] = train5k['output'].apply(remove_before_one)
test2k['output'] = test2k['output'].apply(remove_before_one)
test1k['output'] = test1k['output'].apply(remove_before_one)

# Check
train10k.head()

,input,output
0,[INST] A customer left us a 2-star review: 'th...,1. Acknowledge the issue: Respond to the revie...
1,[INST] A customer left us a 2-star review: 'he...,1. Clear Communication: Ensure that the hotel ...
2,[INST] A customer left us a 2-star review: 'we...,1. Expand and diversify entertainment: Offer a...
3,[INST] A customer left us a 2-star review: 'we...,"e customer's experience, we could focus on mod..."
4,[INST] A customer left us a 2-star review: 'a ...,1. Location: Although we cannot change the phy...


### Initialize the model

### Training